In [62]:
import json
json_path = 'C:/Users/nmizrahi/Downloads/SuperMeatBoy.json'
with open(json_path) as f:
    json_parsed = json.loads(f.read())

In [63]:
EMPTY_NOTE = 0
TICKS_PER_BAR = json_parsed["beatsPerBar"] * json_parsed["ticksPerBeat"]
# 	"beatsPerMinute": 120,

In [66]:
def with_empty_notes(raw_notes):
    dummy_start_note = (EMPTY_NOTE,(0,))
    notes = [dummy_start_note]
    for note in raw_notes:
        try:
            if notes[-1][1][-1] != note[1][0]:
                notes.append((EMPTY_NOTE, (notes[-1][1][-1],note[1][0])))
                print(notes[-2], notes[-1], note)
            notes.append(note)
        except:
            notes.append(note)
    if notes[0] == dummy_start_note:
        notes.pop(0)
    
    return notes

def extract_note_duration(notes):
    return [(note, round(TICKS_PER_BAR/(end-start))) for note, (start,end) in notes]
    
def split_notes(notes):
    separate_notes = [(pitch, list(zip(ticks,ticks[1:]))) for pitch, ticks in notes]
    separate_notes_flat = [(pitch, tick) for pitch, ticks in separate_notes for tick in ticks]
    return separate_notes_flat

def pad_notes_with_stop_note(notes):
        notes.append((EMPTY_NOTE,EMPTY_NOTE))

    
def get_pattern_notes(pattern):
    raw_notes = [(note["pitches"][0],tuple(p["tick"] for p in note["points"])) for note in pattern["notes"]]
    notes = with_empty_notes(raw_notes)
    separate_notes = split_notes(notes)
    notes_with_duration = extract_note_duration(separate_notes)
    pad_notes_with_stop_note(notes_with_duration)
    return notes_with_duration
    
def get_channel_patterns(chan):
    return [get_pattern_notes(pattern) for pattern in chan]
    

In [67]:
channels = [{
        'sequence': chan["sequence"],
        "patterns": get_channel_patterns(chan["patterns"])
    }
    for chan in json_parsed["channels"]]

(0, (0,)) (0, (0, 2)) (31, (2, 4))
(31, (2, 4, 8)) (0, (8, 10)) (36, (10, 12, 16))
(36, (10, 12, 16)) (0, (16, 18)) (41, (18, 20, 24))
(41, (18, 20, 24)) (0, (24, 26)) (43, (26, 28, 32))
(0, (0,)) (0, (0, 2)) (36, (2, 4, 8))
(36, (2, 4, 8)) (0, (8, 10)) (31, (10, 12, 16))
(31, (10, 12, 16)) (0, (16, 18)) (34, (18, 20, 24))
(34, (18, 20, 24)) (0, (24, 26)) (33, (26, 28, 32))
(0, (0,)) (0, (0, 2)) (31, (2, 4))
(0, (0,)) (0, (0, 2)) (48, (2, 3))
(48, (22, 23)) (0, (23, 24)) (48, (24, 25))
(0, (0,)) (0, (0, 4)) (59, (4, 8))
(0, (0,)) (0, (0, 2)) (31, (2, 4, 8))
(31, (2, 4, 8)) (0, (8, 10)) (36, (10, 12, 16))
(36, (10, 12, 16)) (0, (16, 18)) (41, (18, 20, 24))
(41, (18, 20, 24)) (0, (24, 26)) (43, (26, 28, 32))
(0, (0,)) (0, (0, 2)) (36, (2, 4, 8))
(36, (2, 4, 8)) (0, (8, 10)) (31, (10, 12, 16))
(31, (10, 12, 16)) (0, (16, 18)) (34, (18, 20, 24))
(34, (18, 20, 24)) (0, (24, 26)) (33, (26, 28, 32))
(0, (0, 1)) (0, (1, 2)) (10, (2, 3))
(11, (4, 5)) (0, (5, 8)) (0, (8, 10))
(0, (8, 10)) (0, (1

In [68]:
channels

[{'sequence': [1,
   2,
   9,
   2,
   12,
   13,
   1,
   2,
   12,
   14,
   20,
   0,
   15,
   16,
   15,
   17,
   15,
   16,
   15,
   17],
  'patterns': [[(0, 16),
    (31, 16),
    (31, 8),
    (31, 16),
    (36, 16),
    (36, 32),
    (36, 32),
    (36, 16),
    (36, 16),
    (41, 16),
    (41, 16),
    (41, 16),
    (41, 16),
    (43, 16),
    (43, 8),
    (0, 0)],
   [(43, 16),
    (36, 16),
    (36, 8),
    (36, 16),
    (31, 16),
    (31, 8),
    (31, 16),
    (34, 16),
    (34, 8),
    (34, 16),
    (33, 16),
    (33, 8),
    (0, 0)],
   [(43, 16),
    (43, 16),
    (51, 16),
    (51, 16),
    (43, 16),
    (43, 16),
    (51, 16),
    (51, 16),
    (41, 16),
    (41, 16),
    (50, 16),
    (50, 16),
    (41, 16),
    (41, 16),
    (50, 16),
    (50, 16),
    (0, 0)],
   [(39, 8),
    (48, 8),
    (39, 8),
    (48, 8),
    (36, 8),
    (46, 8),
    (36, 8),
    (45, 8),
    (0, 0)],
   [(54, 32),
    (53, 32),
    (31, 16),
    (31, 8),
    (0, 16),
    (36, 16),
    (36, 

In [79]:
def export_channel_to_text(channels):
    seq_len = len(channels[0]['sequence'])
    num_seqs = max(seq for chan in channels for seq in chan['sequence'])
    num_notes = max(len(pattern) for channel in channels for pattern in channel['patterns'])
    return f"""
    const int channel_pattern_sequences<><{seq_len}> PROGMEM = {[channel['sequence'] for channel in channels]};
    
    const note_t channel_note_patterns<><{num_seqs}><{num_notes}> PROGMEM = {[channel['patterns'] for channel in channels]};
    """.replace('[', '\n\t[').replace('[', '{').replace('(', '{').replace(']', '}').replace(')', '}').replace('<', '[').replace('>', ']')
    

In [80]:
print(export_channel_to_text(channels))


    const int channel_pattern_sequences[][20] PROGMEM = 
	{
	{1, 2, 9, 2, 12, 13, 1, 2, 12, 14, 20, 0, 15, 16, 15, 17, 15, 16, 15, 17}, 
	{5, 6, 5, 6, 0, 0, 5, 6, 12, 13, 20, 0, 1, 0, 0, 0, 0, 0, 0, 0}, 
	{0, 0, 0, 0, 0, 6, 5, 5, 5, 6, 7, 7, 0, 5, 5, 6, 5, 6, 5, 6}, 
	{0, 0, 0, 0, 0, 5, 8, 5, 8, 6, 7, 7, 0, 6, 8, 6, 5, 6, 5, 6}};
    
    const note_t channel_note_patterns[][20][33] PROGMEM = 
	{
	{
	{{0, 16}, {31, 16}, {31, 8}, {31, 16}, {36, 16}, {36, 32}, {36, 32}, {36, 16}, {36, 16}, {41, 16}, {41, 16}, {41, 16}, {41, 16}, {43, 16}, {43, 8}, {0, 0}}, 
	{{43, 16}, {36, 16}, {36, 8}, {36, 16}, {31, 16}, {31, 8}, {31, 16}, {34, 16}, {34, 8}, {34, 16}, {33, 16}, {33, 8}, {0, 0}}, 
	{{43, 16}, {43, 16}, {51, 16}, {51, 16}, {43, 16}, {43, 16}, {51, 16}, {51, 16}, {41, 16}, {41, 16}, {50, 16}, {50, 16}, {41, 16}, {41, 16}, {50, 16}, {50, 16}, {0, 0}}, 
	{{39, 8}, {48, 8}, {39, 8}, {48, 8}, {36, 8}, {46, 8}, {36, 8}, {45, 8}, {0, 0}}, 
	{{54, 32}, {53, 32}, {31, 16}, {31, 8}, {0, 16}, {36